In [7]:
import pandas as pd
import gtfs_functions as gtfs
from multiprocessing import  Pool
import numpy as np
import geopandas as gpd

pd.set_option('display.max_rows', 100)

In [8]:
routes2, stops2, stop_times2, trips2, shapes2 = gtfs.import_gtfs('./raw/2022-06-10.zip', busiest_date=False)

In [9]:
updated_10C = trips2[(trips2.shape_id == '1131953846') | (trips2.shape_id == '1131953847') | (trips2.shape_id == '1131953847')]

In [10]:
updated_10C

,trip_id,route_id,service_id,direction_id,shape_id
0,795f79b1-a-23226e03-8,203f5b65,1,1,1131953846
1,fbca78b9-e-23226e03-8,203f5b65,1,0,1131953847
2,0201b206-8-23226e03-8,203f5b65,1,0,1131953847


In [11]:
stop_times_10C = stop_times2.merge(updated_10C, on='trip_id', how='right').sort_values(['trip_id', 'arrival_time'])

In [12]:
stop_times_10C

,trip_id,stop_id,stop_sequence,arrival_time,departure_time,timepoint,shape_dist_traveled,route_id_x,service_id_x,direction_id_x,shape_id_x,stop_code,stop_name,geometry,route_id_y,service_id_y,direction_id_y,shape_id_y
103,0201b206-8-23226e03-8,2234,1,68760.0,68760.0,1,3003.217828,203f5b65,1,0,1131953847,3230,Highway 58 & Oakwood Dr -1-0,POINT (-85.18691 35.08955),203f5b65,1,0,1131953847
104,0201b206-8-23226e03-8,225,2,68820.0,68820.0,0,3566.487227,203f5b65,1,0,1131953847,1895,HIGHWAY 58 + WEBB ROAD,POINT (-85.19172 35.08641),203f5b65,1,0,1131953847
105,0201b206-8-23226e03-8,2233,3,68820.0,68820.0,0,3742.706828,203f5b65,1,0,1131953847,2945,Highway 58 & Trailwood,POINT (-85.19343 35.08567),203f5b65,1,0,1131953847
106,0201b206-8-23226e03-8,302,4,68940.0,68940.0,0,7730.556980,203f5b65,1,0,1131953847,1279,BONNY OAKS + TUGALOO,POINT (-85.23328 35.07451),203f5b65,1,0,1131953847
107,0201b206-8-23226e03-8,301,5,68940.0,68940.0,0,7871.026546,203f5b65,1,0,1131953847,1278,BONNY OAKS + PARKWAY,POINT (-85.23471 35.07403),203f5b65,1,0,1131953847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,fbca78b9-e-23226e03-8,989,44,25740.0,25740.0,0,15744.459787,203f5b65,1,0,1131953847,2243,MCCALLIE AVE + GREENWOOD,POINT (-85.28129 35.03836),203f5b65,1,0,1131953847
99,fbca78b9-e-23226e03-8,990,45,25800.0,25800.0,0,15876.032557,203f5b65,1,0,1131953847,2248,MCCALLIE AVE + HOLTZCLAW,POINT (-85.28261 35.03885),203f5b65,1,0,1131953847
100,fbca78b9-e-23226e03-8,991,46,25800.0,25800.0,0,16078.598334,203f5b65,1,0,1131953847,2261,MCCALLIE AVE + WARNER PK,POINT (-85.28462 35.03963),203f5b65,1,0,1131953847
101,fbca78b9-e-23226e03-8,992,47,25800.0,25800.0,0,16566.326977,203f5b65,1,0,1131953847,2256,MCCALLIE AVE + ONEAL,POINT (-85.28952 35.04139),203f5b65,1,0,1131953847


In [15]:
trips_txt = pd.read_csv('./raw/20211024/trips.txt')
stop_times_txt = pd.read_csv('./raw/20211024/stop_times.txt')
stop_times_txt.trip_id = stop_times_txt.trip_id.astype(str)

In [16]:
trips_txt_10C = pd.read_csv('./raw/20211024/trips.txt')
trips_txt_10C = trips_txt_10C[(trips_txt_10C['route_id'] == '10C')]
trips_txt_10C.trip_id = trips_txt_10C.trip_id.astype(str)

In [17]:
stop_times_txt_10C = stop_times_txt.merge(trips_txt_10C, on='trip_id', how='inner')
# trips_txt_10C

In [18]:
def time_convert(x):
    times = x.split(':')
    return ((3600*int(times[0])+60*int(times[1]))+int(times[2]))

In [19]:
stop_times_txt_10C['arrival_time'] = stop_times_txt_10C['arrival_time'].apply(time_convert)

In [21]:
stop_times_txt_10C.sort_values(['trip_id', 'arrival_time'])[stop_times_txt_10C.stop_sequence == 1].reset_index(drop=True)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,block_name
0,205136020,66300,18:25:00,1940,1,NaN,0,0,0.0,1,10C,1,CAMPBELL-UTC HIGHWAY 58,NaN,0,2402,shp-10C-52,2,2,1519
1,205137020,59760,16:36:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC HIGHWAY 58,NaN,0,2402,shp-10C-51,2,2,1519
2,205138020,57060,15:51:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC HIGHWAY 58,NaN,0,3102,shp-10C-51,2,2,159
3,205139020,57000,15:50:00,1940,1,NaN,0,0,0.0,1,10C,1,CAMPBELL-UTC HIGHWAY 58,NaN,0,7402,shp-10C-52,2,2,905
4,205140020,27000,07:30:00,1940,1,NaN,0,0,0.0,1,10C,1,CAMPBELL-UTC HIGHWAY 58,NaN,0,402,shp-10C-52,2,2,1018
5,205141020,28800,08:00:00,1940,1,NaN,0,0,0.0,1,10C,1,CAMPBELL-UTC HIGHWAY 58,NaN,0,3702,shp-10C-52,2,2,302
6,205142020,47760,13:16:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC HIGHWAY 58,NaN,0,2302,shp-10C-51,2,2,1515
7,205143020,22860,06:21:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC HIGHWAY 58,NaN,0,202,shp-10C-51,2,2,1015
8,205144020,20160,05:36:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC HIGHWAY 58,NaN,0,402,shp-10C-51,2,2,1018
9,205145020,17160,04:46:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC,NaN,0,5202,shp-10C-53,2,2,405


In [25]:
stop_times_txt_10C.stop_id = stop_times_txt_10C.stop_id.astype(str) 
stop_times_txt_10C[['trip_id', 'stop_id', 'arrival_time', 'shape_dist_traveled']].merge(stop_times_10C, on=['stop_id', 'arrival_time'], how='inner')

,trip_id_x,stop_id,arrival_time,shape_dist_traveled_x,trip_id_y,stop_sequence,departure_time,timepoint,shape_dist_traveled_y,route_id_x,service_id_x,direction_id_x,shape_id_x,stop_code,stop_name,geometry,route_id_y,service_id_y,direction_id_y,shape_id_y
